# Cosmos PoC

A Colab runtime with GPU acceleration is required. If you're using a CPU-only runtime, you can switch using the menu "Runtime > Change runtime type".

# Installing dm_control

In [ ]:
#@title Run to install MuJoCo and `dm_control`
import distutils.util
import subprocess
if subprocess.run('nvidia-smi').returncode:
  raise RuntimeError(
      'Cannot communicate with GPU. '
      'Make sure you are using a GPU Colab runtime. '
      'Go to the Runtime menu and select Choose runtime type.')

print('Installing dm_control...')
!pip install -q dm_control>=1.0.7

# Configure dm_control to use the EGL rendering backend (requires GPU)
%env MUJOCO_GL=egl

print('Checking that the dm_control installation succeeded...')
try:
  from dm_control import suite
  env = suite.load('cartpole', 'swingup')
  pixels = env.physics.render()
except Exception as e:
  raise e from RuntimeError(
      'Something went wrong during installation. Check the shell output above '
      'for more information.\n'
      'If using a hosted Colab runtime, make sure you enable GPU acceleration '
      'by going to the Runtime menu and selecting "Choose runtime type".')
else:
  del pixels, suite

!echo Installed dm_control $(pip show dm_control | grep -Po "(?<=Version: ).+")

# Imports

Run both of these cells:

In [ ]:
#@title All `dm_control` imports required for this tutorial

# The basic mujoco wrapper.
from dm_control import mujoco

# Access to enums and MuJoCo library functions.
from dm_control.mujoco.wrapper.mjbindings import enums
from dm_control.mujoco.wrapper.mjbindings import mjlib

# PyMJCF
from dm_control import mjcf

# Composer high level imports
from dm_control import composer
from dm_control.composer.observation import observable
from dm_control.composer import variation

# Imports for Composer tutorial example
from dm_control.composer.variation import distributions
from dm_control.composer.variation import noises
from dm_control.locomotion.arenas import floors

# Control Suite
from dm_control import suite

# Run through corridor example
from dm_control.locomotion.walkers import cmu_humanoid
from dm_control.locomotion.arenas import corridors as corridor_arenas
from dm_control.locomotion.tasks import corridors as corridor_tasks

# Soccer
from dm_control.locomotion import soccer

# Manipulation
from dm_control import manipulation

In [ ]:
#@title Other imports and helper functions

# General
import copy
import os
import itertools
from IPython.display import clear_output
import numpy as np

# Graphics-related
import matplotlib
import matplotlib.animation as animation
import matplotlib.pyplot as plt
from IPython.display import HTML
import PIL.Image
# Internal loading of video libraries.

# Use svg backend for figure rendering
%config InlineBackend.figure_format = 'svg'

# Font sizes
SMALL_SIZE = 8
MEDIUM_SIZE = 10
BIGGER_SIZE = 12
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

# Inline video helper function
if os.environ.get('COLAB_NOTEBOOK_TEST', False):
  # We skip video generation during tests, as it is quite expensive.
  display_video = lambda *args, **kwargs: None
else:
  def display_video(frames, framerate=30):
    height, width, _ = frames[0].shape
    dpi = 70
    orig_backend = matplotlib.get_backend()
    matplotlib.use('Agg')  # Switch to headless 'Agg' to inhibit figure rendering.
    fig, ax = plt.subplots(1, 1, figsize=(width / dpi, height / dpi), dpi=dpi)
    matplotlib.use(orig_backend)  # Switch back to the original backend.
    ax.set_axis_off()
    ax.set_aspect('equal')
    ax.set_position([0, 0, 1, 1])
    im = ax.imshow(frames[0])
    def update(frame):
      im.set_data(frame)
      return [im]
    interval = 1000/framerate
    anim = animation.FuncAnimation(fig=fig, func=update, frames=frames,
                                   interval=interval, blit=True, repeat=False)
    return HTML(anim.to_html5_video())

# Seed numpy's global RNG so that cell outputs are deterministic. We also try to
# use RandomState instances that are local to a single cell wherever possible.
np.random.seed(42)

# Planet orbiting around a "sun"

In [ ]:
#@title The orbit model{vertical-output: true}

orbit = """
<mujoco model="orbit">
  <option integrator="RK4"/>
  <worldbody>
    <light pos="0 0 .6"/>
    <camera name="closeup" pos="0 -.1 .07" xyaxes="1 0 0 0 1 2"/>
    <body name="top" pos="0 0 .02">
      <freejoint/>
      <geom name="ball" type="sphere" size=".02" rgba="1 .62 .05 1"/>
      <geom name="stem" type="sphere" pos=".03 0 0" size="0.004" rgba=".43 .89 .96 1"/>
      <geom name="ballast" type="box" size=".023 .023 0.005"  pos="0 0 -.015" 
       contype="0" conaffinity="0" group="3"/>
    </body>
  </worldbody>
  <option>
    <flag gravity="disable"/>
  </option>
  <keyframe>
    <key name="spinning" qpos="0 0 0.02 1 0 0 0" qvel="0 0 0 0 0 2.5" />
  </keyframe>
</mujoco>
"""
physics_orbit = mujoco.Physics.from_xml_string(orbit)
PIL.Image.fromarray(physics_orbit.render(camera_id='closeup'))

In [ ]:
#@title Video of the orbit {vertical-output: true}

duration = 7    # (seconds)
framerate = 60  # (Hz)

# Simulate and display video.
frames = []
physics_orbit.reset(0)  # Reset to keyframe 0 (load a saved state).
while physics_orbit.data.time < duration:
  physics_orbit.step()
  if len(frames) < (physics_orbit.data.time) * framerate:
    pixels = physics_orbit.render(camera_id='closeup')
    frames.append(pixels)

display_video(frames, framerate)

# Falling ball

In [ ]:
#@title The falling ball model{vertical-output: true}

falling_ball = """
<mujoco model="falling ball">
  <option integrator="RK4"/>
  <asset>
    <texture name="grid" type="2d" builtin="checker" rgb1=".1 .2 .3" 
     rgb2=".2 .3 .4" width="300" height="300"/>
    <material name="grid" texture="grid" texrepeat="8 8" reflectance=".2"/>
  </asset>
  <visual>
    <map stiffness="0" force="100"/>
  </visual>
  <worldbody>
    <geom size=".2 .2 .01" type="plane" material="grid"/>
    <light pos="0 0 .6"/>
    <camera name="closeup" pos="0 -.1 .07" xyaxes="1 0 0 0 1 2"/>
    <body name="top" pos="0 0 .02">
      <joint name="ball" type="free" pos="0 0 0" stiffness=".073"/>
      <geom type="sphere" pos="0 0 .02" size="0.004"/>
    </body>
  </worldbody>
  <option>
    <flag refsafe="disable"/>
  </option>
</mujoco>
"""
physics_ball = mujoco.Physics.from_xml_string(falling_ball)
PIL.Image.fromarray(physics_ball.render(camera_id='closeup'))

In [ ]:
#@title Video of the falling ball {vertical-output: true}

duration = 7    # (seconds)
framerate = 60  # (Hz)

# Simulate and display video.
frames = []
# physics_ball.reset(0)  # Reset to keyframe 0 (load a saved state).
while physics_ball.data.time < duration:
  physics_ball.step()
  if len(frames) < (physics_ball.data.time) * framerate:
    pixels = physics_ball.render(camera_id='closeup')
    frames.append(pixels)

display_video(frames, framerate)

# Falling ball with smaller gravity but same stiffness

In [ ]:
#@title The falling ball model with smaller gravity but same stiffness{vertical-output: true}

falling_ball = """
<mujoco model="falling ball">
  <option integrator="RK4"/>
  <asset>
    <texture name="grid" type="2d" builtin="checker" rgb1=".1 .2 .3" 
     rgb2=".2 .3 .4" width="300" height="300"/>
    <material name="grid" texture="grid" texrepeat="8 8" reflectance=".2"/>
  </asset>
  <visual>
    <map stiffness="0" force="100"/>
  </visual>
  <worldbody>
    <geom size=".2 .2 .01" type="plane" material="grid"/>
    <light pos="0 0 .6"/>
    <camera name="closeup" pos="0 -.1 .07" xyaxes="1 0 0 0 1 2"/>
    <body name="top" pos="0 0 .02">
      <joint name="ball" type="free" pos="0 0 0" stiffness=".073"/>
      <geom type="sphere" pos="0 0 .02" size="0.004"/>
    </body>
  </worldbody>
  <option gravity="0 0 -5">
    <flag refsafe="disable"/>
  </option>
</mujoco>
"""
physics_ball = mujoco.Physics.from_xml_string(falling_ball)
PIL.Image.fromarray(physics_ball.render(camera_id='closeup'))

In [ ]:
#@title Video of the falling ball with smaller gravity but stame stiffness {vertical-output: true}

duration = 7    # (seconds)
framerate = 60  # (Hz)

# Simulate and display video.
frames = []
# physics_ball.reset(0)  # Reset to keyframe 0 (load a saved state).
while physics_ball.data.time < duration:
  physics_ball.step()
  if len(frames) < (physics_ball.data.time) * framerate:
    pixels = physics_ball.render(camera_id='closeup')
    frames.append(pixels)

display_video(frames, framerate)

# Falling ball with very small gravity and no stiffness

In [ ]:
#@title The falling ball model with very small gravity and no stiffness {vertical-output: true}

falling_ball = """
<mujoco model="falling ball">
  <option integrator="RK4"/>
  <asset>
    <texture name="grid" type="2d" builtin="checker" rgb1=".1 .2 .3" 
     rgb2=".2 .3 .4" width="300" height="300"/>
    <material name="grid" texture="grid" texrepeat="8 8" reflectance=".2"/>
  </asset>
  <visual>
    <map stiffness="0" force="100"/>
  </visual>
  <worldbody>
    <geom size=".2 .2 .01" type="plane" material="grid"/>
    <light pos="0 0 .6"/>
    <camera name="closeup" pos="0 -.1 .07" xyaxes="1 0 0 0 1 2"/>
    <body name="top" pos="0 0 .02">
      <joint name="ball" type="free" pos="0 0 0"/>
      <geom type="sphere" pos="0 0 .02" size="0.004"/>
    </body>
  </worldbody>
  <option gravity="0 0 -.1">
    <flag refsafe="disable"/>
  </option>
</mujoco>
"""
physics_ball = mujoco.Physics.from_xml_string(falling_ball)
PIL.Image.fromarray(physics_ball.render(camera_id='closeup'))

In [ ]:
#@title Video of the falling ball with very small gravity and no stiffness {vertical-output: true}

duration = 7    # (seconds)
framerate = 60  # (Hz)

# Simulate and display video.
frames = []
# physics_ball.reset(0)  # Reset to keyframe 0 (load a saved state).
while physics_ball.data.time < duration:
  physics_ball.step()
  if len(frames) < (physics_ball.data.time) * framerate:
    pixels = physics_ball.render(camera_id='closeup')
    frames.append(pixels)

display_video(frames, framerate)

# Throwing a ball with a vertical velocity in the X direction

In [ ]:
#@title The throwing ball model{vertical-output: true}

ball_throwing = """
<mujoco model="ball throwing">
  <option integrator="RK4"/>
  <asset>
    <texture name="grid" type="2d" builtin="checker" rgb1=".1 .2 .3" 
     rgb2=".2 .3 .4" width="300" height="300"/>
    <material name="grid" texture="grid" texrepeat="8 8" reflectance=".2"/>
  </asset>
  <worldbody>
    <geom size=".2 .2 .01" type="plane" material="grid"/>
    <light pos="0 0 .6"/>
    <camera name="closeup" pos="0 -.1 .07" xyaxes="1 0 0 0 1 2"/>
    <body name="top" pos="0 0 .02">
      <joint name="ball" type="free" pos="0 0 0" stiffness=".003"/>
      <geom name="ball" type="sphere" size=".004" pos="-.04 0 .02"/>
    </body>
  </worldbody>
  <keyframe>
    <key name="spinning" qpos="0 0 0.02 0 0 0 0" qvel=".5 0 0 0 0 0" />
  </keyframe>
</mujoco>
"""
physics = mujoco.Physics.from_xml_string(ball_throwing)
PIL.Image.fromarray(physics.render(camera_id='closeup'))

In [ ]:
#@title Video of throwing a ball with a vertical velocity {vertical-output: true}

duration = 1    # (seconds)
framerate = 60  # (Hz)

# Simulate and display video.
frames = []
physics.reset(0)  # Reset to keyframe 0 (load a saved state).
while physics.data.time < duration:
  physics.step()
  if len(frames) < (physics.data.time) * framerate:
    pixels = physics.render(camera_id='closeup')
    frames.append(pixels)

frames = frames + frames + frames

display_video(frames, framerate)

# Throwing a ball verically with -5 gravity



In [ ]:
#@title Throwing a ball vertically with -5 gravity{vertical-output: true}

ball_throwing = """
<mujoco model="ball throwing">
  <option integrator="RK4"/>
  <asset>
    <texture name="grid" type="2d" builtin="checker" rgb1=".1 .2 .3" 
     rgb2=".2 .3 .4" width="300" height="300"/>
    <material name="grid" texture="grid" texrepeat="8 8" reflectance=".2"/>
  </asset>
  <worldbody>
    <geom size=".2 .2 .01" type="plane" material="grid"/>
    <light pos="0 0 .6"/>
    <camera name="closeup" pos="0 -.1 .07" xyaxes="1 0 0 0 1 2"/>
    <body name="top" pos="0 0 .02">
      <joint name="ball" type="free" pos="0 0 0" stiffness=".003"/>
      <geom name="ball" type="sphere" size=".004" pos="-.04 0 .02"/>
    </body>
  </worldbody>
  <keyframe>
    <key name="spinning" qpos="0 0 0.02 0 0 0 0" qvel=".5 0 0 0 0 0" />
  </keyframe>
  <option gravity="0 0 -5">
    <flag refsafe="disable"/>
  </option>
</mujoco>
"""
physics = mujoco.Physics.from_xml_string(ball_throwing)
PIL.Image.fromarray(physics.render(camera_id='closeup'))

In [ ]:
#@title Video of throwing a ball vertically with -5 gravity {vertical-output: true}

duration = 1    # (seconds)
framerate = 60  # (Hz)

# Simulate and display video.
frames = []
physics.reset(0)  # Reset to keyframe 0 (load a saved state).
while physics.data.time < duration:
  physics.step()
  if len(frames) < (physics.data.time) * framerate:
    pixels = physics.render(camera_id='closeup')
    frames.append(pixels)

frames = frames + frames + frames

display_video(frames, framerate)

# Throwing a ball verically with -1 gravity

In [ ]:
#@title Throwing a ball vertically with -1 gravity{vertical-output: true}

ball_throwing = """
<mujoco model="ball throwing">
  <option integrator="RK4"/>
  <asset>
    <texture name="grid" type="2d" builtin="checker" rgb1=".1 .2 .3" 
     rgb2=".2 .3 .4" width="300" height="300"/>
    <material name="grid" texture="grid" texrepeat="8 8" reflectance=".2"/>
  </asset>
  <worldbody>
    <geom size=".2 .2 .01" type="plane" material="grid"/>
    <light pos="0 0 .6"/>
    <camera name="closeup" pos="0 -.1 .07" xyaxes="1 0 0 0 1 2"/>
    <body name="top" pos="0 0 .02">
      <joint name="ball" type="free" pos="0 0 0" stiffness=".003"/>
      <geom name="ball" type="sphere" size=".004" pos="-.04 0 .02"/>
    </body>
  </worldbody>
  <keyframe>
    <key name="spinning" qpos="0 0 0.02 0 0 0 0" qvel=".4 0 0 0 0 0" />
  </keyframe>
  <option gravity="0 0 -1">
    <flag refsafe="disable"/>
  </option>
</mujoco>
"""
physics = mujoco.Physics.from_xml_string(ball_throwing)
PIL.Image.fromarray(physics.render(camera_id='closeup'))

In [ ]:
#@title Video of throwing a ball vertically with -5 gravity {vertical-output: true}

duration = 1    # (seconds)
framerate = 60  # (Hz)

# Simulate and display video.
frames = []
physics.reset(0)  # Reset to keyframe 0 (load a saved state).
while physics.data.time < duration:
  physics.step()
  if len(frames) < (physics.data.time) * framerate:
    pixels = physics.render(camera_id='closeup')
    frames.append(pixels)

frames = frames + frames + frames

display_video(frames, framerate)